In [ ]:
# Imports

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

In [ ]:
import os
os.chdir("/home2/jgcw74/l3_project")  # go to project root
import importlib

import helpers
import dataset_processing
import models
import xai

torch_device = helpers.utils.get_torch_device()

In [ ]:
import typing as t
from pathlib import Path
import json

import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
AVAILABLE_MODELS = ("ConvNeXtSmall", "ResNet50", "SwinTransformerSmall")

# Helper functions

In [ ]:
def get_dataset_and_model(dataset_n: dataset_processing.DATASET_NAMES, model_n: models.MODEL_NAMES):
    model_type = models.get_model_type(model_n)
    ds = dataset_processing.get_dataset_object(dataset_n, "test", model_type.expected_input_dim, 32, 4, torch_device)

    m = model_type(False, ds.N_BANDS, ds.N_CLASSES).to(torch_device)
    weights_path = json.load(Path("weights_paths.json").open("r"))[dataset_n][model_n]
    m.load_weights(Path("checkpoints") / dataset_n / model_n / weights_path)

    return ds, m

# Figure 1 - KPCACAM on EuroSATMS

In [ ]:
dataset_name = "EuroSATMS"
dataset, model = get_dataset_and_model(dataset_name, "ResNet50")
print(list(enumerate(dataset.classes)))

In [ ]:
class_idx = 8
batch_num = 0
kpca_exp = xai.get_explainer_object(
    "KPCACAM", model,
    extra_path=Path(dataset_name) / f"c{class_idx:02}" / f"b{batch_num:03}",
)
kpca_exp.force_load()

In [ ]:
importlib.reload(helpers.plotting)
helpers.plotting.show_image(kpca_exp.input[:8])

In [ ]:
helpers.plotting.visualise_importance(kpca_exp.input.numpy(force=True)[:8], kpca_exp.explanation[:8], alpha=0.5, with_colorbar=False, band_idxs=dataset.rgb_indices)

In [ ]:
randomised_exp = xai.get_explainer_object(
    "KPCACAM", model,
    extra_path=Path(dataset_name) / f"c{class_idx:02}" / "combined" / "randomised",
)
randomised_exp.force_load()

In [ ]:
helpers.plotting.visualise_importance(randomised_exp.input.numpy(force=True)[:8], randomised_exp.explanation[:8], alpha=0.5, with_colorbar=False, band_idxs=dataset.rgb_indices)

Extremely similar explanations to those provided for the un-randomised model.